In [1]:
import sys
sys.path.append("..")
from allennlp.predictors.predictor import Predictor
import json, stanza, pickle, datetime, time, re, os, random, pymongo, csv, copy,  \
allennlp_models.structured_prediction
from multiprocessing import Process, Manager, Pool
from pymongo import MongoClient
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from pylab import mpl

/root/miniconda3/envs/hald/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MongoClient(username="hald",password="mclab236")

In [3]:
now=datetime.datetime.now()

In [4]:
now_time=now.strftime('%Y_%m_%d')

In [5]:
now_time='2023_09_30'

In [6]:
pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [7]:
db = client.hald

In [8]:
Gene_entities=list(db.Gene_raw.find({'addtime':now_time}))
RNA_entities=list(db.RNA_raw.find({'addtime':now_time}))
Carbohydrate_entities=list(db.Carbohydrate_raw.find({'addtime':now_time}))
Lipid_entities=list(db.Lipid_raw.find({'addtime':now_time}))
Peptide_entities=list(db.Peptide_raw.find({'addtime':now_time}))
Pharmaceutical_Preparations_entities=list(db.Pharmaceutical_Preparations_raw.find({'addtime':now_time}))
Protein_entities=list(db.Protein_raw.find({'addtime':now_time}))
Toxin_entities=list(db.Toxin_raw.find({'addtime':now_time}))

In [9]:
Mutation_entities=[]
Disease_entities=[]
for i in list(db.Mutation_raw.find({'addtime':now_time})):
    if i.get('Genomic View'):
        Mutation_entities.append(i)
for i in list(db.Disease_raw.find({'addtime':now_time})):
    if i.get('url'):
        Disease_entities.append(i)

In [10]:
names=locals()

In [11]:
for i in pairs:
    print(len(names[i+'_entities']))

18982
339
2748
2372
731
57
600
126
945
173979


In [12]:
for j in pairs:
    names[j+'_dict']={}
    for i in names[j+'_entities']:
        names[j+'_dict'][i.get('PMID')]=names[j+'_dict'].get(i.get('PMID'),[])+[i]

In [13]:
diff_relation_dict={}
for j in range(len(pairs)-1):
    for i in range(j+1,len(pairs)):
        for k in list(set(names[pairs[j]+'_dict'].keys()).intersection(set(names[pairs[i]+'_dict'].keys()))):
            for l in names[pairs[j]+'_dict'].get(k):
                for m in names[pairs[i]+'_dict'].get(k):
                    if l.get('sentence')==m.get('sentence') and l.get('entity')!=m.get('entity'):
                        diff_relation_dict[l.get('sentence')]=diff_relation_dict.get(l.get('sentence'),[])+[{'PMID':l.get('PMID'),
                                                                                                  'entity1':l.get('entity'),
                                                                                                   'entity2':m.get('entity'),
                                                                                                   'sentence':l.get('sentence'),
                                                                                                   'AB':l.get('AB'),
                                                                                                   'target1':l.get('target'),
                                                                                                    'target1_type':l.get('type'),
                                                                                                   'target2':m.get('target'),
                                                                                                    'target2_type':m.get('type'),
                                                                                                   'PMID':l.get('PMID'),
                                                                                                  }]

In [14]:
same_relation_dict={}
for j in range(len(pairs)):
    for k in names[pairs[j]+'_dict'].values():
        for l in range(len(k)-1):
            for m in range(l+1,len(k)):
                if k[l].get('sentence')==k[m].get('sentence'):
                    if k[l].get('entity')!=k[m].get('entity'):
                        same_relation_dict[k[l].get('sentence')]=same_relation_dict.get(k[l].get('sentence'),[])+[{'PMID':k[l].get('PMID'),
                                                                                                  'entity1':k[l].get('entity'),
                                                                                                   'entity2':k[m].get('entity'),
                                                                                                   'sentence':k[l].get('sentence'),
                                                                                                   'AB':k[l].get('AB'),
                                                                                                   'target1':k[l].get('target'),
                                                                                                    'target1_type':k[l].get('type'),
                                                                                                   'target2':k[m].get('target'),
                                                                                                    'target2_type':k[l].get('type'),
                                                                                                   'PMID':k[l].get('PMID'),
                                                                                                  }]

In [15]:
print(len(same_relation_dict))
print(len(diff_relation_dict))
print(len(list(set(list(same_relation_dict.keys())).intersection(set(list(diff_relation_dict.keys()))))))

35508
7625
2815


In [16]:
relation_dict={}
for i in same_relation_dict.keys():
    if diff_relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)+diff_relation_dict.get(i)

In [17]:
for i in same_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)
print(len(relation_dict))
for i in diff_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=diff_relation_dict.get(i)
print(len(relation_dict))

35508
40318


In [18]:
list(relation_dict.items())[-1]

('These results provide evidence that N-acetylcysteine inhibits thrombosis in JAK2V617F mice and provide a pre-clinical rationale for investigating NAC as a therapeutic to reduce thrombotic risk in MPN.',
 [{'PMID': '31978215',
   'entity1': 'rs77375493',
   'entity2': 'Thrombosis',
   'sentence': 'These results provide evidence that N-acetylcysteine inhibits thrombosis in JAK2V617F mice and provide a pre-clinical rationale for investigating NAC as a therapeutic to reduce thrombotic risk in MPN.',
   'AB': 'Thrombosis is a major cause of mortality in patients with myeloproliferative neoplasms (MPNs), though there is currently little to offer patients with MPN beyond aspirin and cytoreductive therapies such as hydroxyurea for primary prevention. Thrombogenesis in MPN involves multiple cellular mechanisms, including platelet activation and neutrophil-extracellular trap formation; therefore, an antithrombotic agent that targets one or more of these processes would be of therapeutic benefi

In [19]:
relation_list=[]
for i in relation_dict.values():
    for j in i:
        relation_list.append(j)

In [20]:
colons=[]
not_colons=[]
for i in relation_list:
    if not re.search(':',i.get('sentence')[:50],flags=re.M|re.I):
        if re.search('\snot\s',i.get('sentence'),flags=re.M|re.I):
            not_colons.append(i)

In [21]:
len(not_colons)

3849

In [22]:
len(relation_list)

106888

In [23]:
not_colons[-1]

{'PMID': '30092269',
 'entity1': 'rs730880031',
 'entity2': 'Mitochondrial Diseases',
 'sentence': 'We also show that G66V fibroblasts do not display the loss of MICOS complex integrity and mitochondrial damage found in S59L cells.',
 'AB': 'Following the involvement of CHCHD10 in FrontoTemporal-Dementia-Amyotrophic Lateral Sclerosis (FTD-ALS) clinical spectrum, a founder mutation (p.Gly66Val) in the same gene was identified in Finnish families with late-onset spinal motor neuronopathy (SMAJ). SMAJ is a slowly progressive form of spinal muscular atrophy with a life expectancy within normal range. In order to understand why the p.Ser59Leu mutation, responsible for severe FTD-ALS, and the p.Gly66Val mutation could lead to different levels of severity, we compared their effects in patient cells. Unlike affected individuals bearing the p.Ser59Leu mutation, patients presenting with SMAJ phenotype have neither mitochondrial myopathy nor mtDNA instability. The expression of CHCHD10S59L mutant

In [24]:
colons=[]
triplelist_filtered=[]
for i in relation_list:
    if re.search(':',i.get('sentence')[:50],flags=re.M|re.I):
        colons.append(copy.deepcopy(i))
    else:
        triplelist_filtered.append(copy.deepcopy(i))

In [25]:
print(len(relation_list))
print(len(colons))
print(len(triplelist_filtered))

106888
22430
84458


In [26]:
colons[0]

{'PMID': '37201763',
 'entity1': 'SLC19A1',
 'entity2': 'MTHFR',
 'sentence': 'CONCLUSIONS: There was association between folate pathway gene polymorphism of Solute Carrier Family 19 Member 1 (SLC19A1) and Methylenetetrahydrofolate Reductase (MTHFR) with erythrocyte SOD and GSH-PX activities, and folate levels in male but not in female aging subjects.',
 'AB': "BACKGROUND: Plasma folate levels are closely related to antioxidant capacity and are regulated by folate pathway gene polymorphism. However, few studies have explored the gender-specific association of folate pathway gene polymorphism with oxidative stress biomarkers. The present study was designed to explore the gender-specific independent and combined impacts of solute carrier family 19 member 1 (SLC19A1) and methylenetetrahydrofolate reductase (MTHFR) genetic polymorphisms on oxidative stress biomarkers in older adults. METHODS: A total of 401 subjects were recruited, including 145 males and 256 females. Demographic character

In [52]:
sentences_with_colons=[]
sentences_with_colons_set=set()
sentences_without_colons=[]
sentences_with_colons_dict={}
sentences_without_colons_dict={}
for i in colons:
    sentences_with_colons_dict[i.get('sentence')]=sentences_with_colons_dict.get(i.get('sentence'),[])+[copy.deepcopy(i)]
    if not re.search(r'(result(s)?: )|(objective(s)?: )|(method(s)?: )|(background(s)?: )|(purpose(s)?: )|(conclusion(s)?: )|(introduction(s)?: )|(participants(s)?: )|(aim(s)?: )|(importance(s)?: )|(discussion(s)?: )|(abstract(s)?: )|(recent finding(s)?: )|([A-Z]+: )|([A-Z]+\(S\): )|(([A-Z]+\s(&\s)?)*[A-Z]+: )|(([A-Z]+\/)*[A-Z]+: )',i.get('sentence')[:50],flags=re.M|re.I):
        sentences_with_colons.append(i.get('sentence'))
        sentences_with_colons_set.add(i.get('sentence'))
    else:
        sentences_without_colons.append(re.sub(r'.*(result(s)?: )|(objective(s)?: )|(method(s)?: )|(background(s)?: )|(purpose(s)?: )|(conclusion(s)?: )|(introduction(s)?: )|(participants(s)?: )|(aim(s)?: )|(importance(s)?: )|(discussion(s)?: )|(abstract(s)?: )|(recent finding(s)?: )|([A-Z]+: )|([A-Z]+\(S\): )|(([A-Z]+\s(&\s)?)*[A-Z]+: )|(([A-Z]+\/)*[A-Z]+: )','',copy.deepcopy(i).get('sentence'),flags=re.M|re.I))

In [53]:
len(sentences_without_colons)

22203

In [54]:
len(sentences_with_colons)

227

## List sentences_with_colons_set and define sentences_with_colons_filtered with normalized sentences!!!

In [55]:
# Need to redefine!!!
sentences_with_colons_filtered=[]

In [56]:
sentences_with_colons_filtered=sentences_with_colons_filtered+sentences_without_colons

In [57]:
len(sentences_with_colons_filtered)

22203

In [58]:
sentence_colons={}
for i in sentences_with_colons_filtered:
    for j,k in sentences_with_colons_dict.items():
        if i[-30:-1]==j[-30:-1]:
            for l in k:
                l.update({'sentence_transformed':i})
            sentence_colons[i]=k

In [59]:
len(sentence_colons)

8734

In [60]:
len(sentences_with_colons_dict)

8834

In [61]:
triplelist_filtered[0]

{'PMID': '37766191',
 'entity1': 'TLR4',
 'entity2': 'TLR2',
 'sentence': 'Moreover, individuals carrying either the TLR2-rs5743708 or the TLR4-rs4986791 polymorphisms exhibited a 3.6- and 2.5-fold increased probability for developing pneumonia and a more severe disease, respectively.',
 'AB': "COVID-19 is characterized by a heterogeneous clinical presentation and prognosis. Risk factors contributing to the development of severe disease include old age and the presence of comorbidities. However, the genetic background of the host has also been recognized as an important determinant of disease prognosis. Considering the pivotal role of innate immunity in the control of SARS-CoV-2 infection, we analyzed the possible contribution of several innate immune gene polymorphisms (including TLR2-rs5743708, TLR4-rs4986790, TLR4-rs4986791, CD14-rs2569190, CARD8-rs1834481, IL18-rs2043211, and CD40-rs1883832) in disease severity and prognosis. A total of 249 individuals were enrolled and further div

In [62]:
all_relations=copy.deepcopy(sentence_colons)
for i in triplelist_filtered:
    i.update({'sentence_transformed':i.get('sentence')})
    all_relations[i.get('sentence')]=all_relations.get(i.get('sentence'),[])+[i]

In [63]:
len(all_relations)

40214

In [64]:
all_relations_list=list(all_relations.keys())

# Deal with all

## Prepare for OpenIE

In [65]:
os.makedirs('results/step3/openie/filelist/')

In [66]:
ind=1
for i in all_relations_list:
    f=open('results/step3/openie/filelist/sentence'+str(ind)+'.txt','w')
    f.write(i)
    f.close()
    ind+=1

In [67]:
f=open('results/step3/openie/filelist.txt','a')
for i in range(1,len(all_relations_list)+1):
    f.write('filelist/sentence'+str(i)+'.txt'+'\n')
f.close()

### Run in shell 

In [40]:
# cd results/step3/openie
# nohup java -mx500g -cp ../../../../stanza_corenlp/stanford-corenlp-4.3.2.jar:../../../../stanza_corenlp/stanford-corenlp-4.3.2-models.jar:../../../../stanza_corenlp/CoreNLP-to-HTML.xsl:../../../../slf4j-api.jar:../../../../stanza_corenlp/slf4j-simple.jar edu.stanford.nlp.naturalli.OpenIE -format reverb -filelist ./filelist.txt -output ./openie.txt > ./openie.log &
# cd ../../../

In [72]:
len(all_relations_list)

40214

## Prepare for Stanza

In [68]:
os.makedirs('results/step3/openie/files/')
k=1
for i in range(0,len(all_relations_list),len(all_relations_list)//4):
    pickle.dump(all_relations_list[i:i+len(all_relations_list)//4],open('results/step3/openie/files/openie'+str(k)+'.pkl','wb'))
    k+=1

In [69]:
os.makedirs('results/step3/rules/files')
k=1
for i in range(0,len(all_relations_list),len(all_relations_list)//4):
    f=open('results/step3/rules/files/rules'+str(k)+'.py','w')
    f.write('''
import stanza
import os
import datetime
import time
nlp = stanza.Pipeline('en', package='genia',download_method=None)
import pickle
raw=pickle.load(open('../../openie/files/openie'''+str(k)+'''.pkl','rb'))
import re 

def nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbtype,sentence2adv):
    sentence=i
    doc=nlp(sentence)
    words=[]
    for sent in doc.sentences:
        for word in sent.words:
            words.append(word.text)
    relations=[]
    verblist=[]
    verbprototype=[]
    adv=[]
    for sent in doc.sentences:
        for word in sent.words:
            relations.append([word.text,words[int(word.head)-1],int(word.id),int(word.head),word.deprel,word.xpos])
            if word.xpos in ['VB','VBD','VBG','VBN', 'VBP', 'VBZ']:
                verblist.append({word.text:word.xpos})
                verbprototype.append({word.text:word.lemma})
            if word.xpos in ['RB','RBR','RBS']:
                adv.append({word.text:word.xpos})
    sentence2words[i]=words
    sentence2relations[i]=relations
    sentence2verblist[i]=verblist
    sentence2verbprototype[i]=verbprototype
    sentence2adv[i]=adv


start=datetime.datetime.now()
print('Parent process %s.' % os.getpid())
sentence2words={}
sentence2relations={}
sentence2verblist={}
sentence2verbprototype={}
sentence2adv={}
for i in raw:
    nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbprototype,sentence2adv)
pickle.dump(sentence2words,open('sentence2words'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2relations,open('sentence2relations'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verblist,open('sentence2verblist'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verbprototype,open('sentence2verbprototype'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2adv,open('sentence2adv'''+str(k)+'''.pkl','wb'))
print('Waiting for all subprocesses done...')

end=datetime.datetime.now()
print("The running time is "+str((end-start).seconds)+"s")  
    ''')
    f.close()
    k+=1

In [80]:
import pickle
raw=pickle.load(open('results/step3/openie/files/openie'+str(1)+'.pkl','rb'))

### Run in shell 

In [111]:
# cd results/step3/rules/files
# for i in {1..5}; do echo "nohup python rules${i}.py >> rules.log &" | bash; done
# cd ../../../../

## Deal with OpenIE

In [82]:
openie_table=pd.read_table('results/step3/openie/openie.txt',header=None)

In [83]:
openie_table.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,filelist/sentence105.txt,0,polymorphisms,is in,ApoE,1,2,2,3,3,4,1.0,The polymorphisms in ApoE and IL -6-174 G / C ...,DT NNS IN NN CC NN NN NN HYPH NN NNS VBD VBN V...,polymorphism,be in,apoe
1,filelist/sentence105.txt,0,polymorphisms,were,assessed,1,2,11,12,12,13,1.0,The polymorphisms in ApoE and IL -6-174 G / C ...,DT NNS IN NN CC NN NN NN HYPH NN NNS VBD VBN V...,polymorphism,be,assess
2,filelist/sentence90.txt,0,Lapatinib,is,dual,0,1,1,2,3,4,1.0,Lapatinib is a dual EGFR and HER2 inhibitor th...,NNP VBZ DT JJ NN CC NN NN WDT VBZ VBN TO VB NN...,Lapatinib,be,dual
3,filelist/sentence90.txt,0,Lapatinib,is,EGFR,0,1,1,2,4,5,1.0,Lapatinib is a dual EGFR and HER2 inhibitor th...,NNP VBZ DT JJ NN CC NN NN WDT VBZ VBN TO VB NN...,Lapatinib,be,egfr
4,filelist/sentence90.txt,0,Lapatinib,is,dual EGFR,0,1,1,2,3,5,1.0,Lapatinib is a dual EGFR and HER2 inhibitor th...,NNP VBZ DT JJ NN CC NN NN WDT VBZ VBN TO VB NN...,Lapatinib,be,dual egfr


In [84]:
len(openie_table)

657647

In [85]:
df=openie_table.loc[:,[0,2,3,4,12]]

In [86]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'

In [87]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']
for i in rules_pairs:
    names[i]={}
    for k in ['results/step3/rules/files/'+i+str(j)+'.pkl' for j in range(1,6)]:
        names[i].update(pickle.load(open(k,'rb')))

In [88]:
finallist=[]
for line in range(len(df)):
    i=all_relations_list[int(df.iloc[line,0][17:-4])-1]
    verblist=sentence2verblist.get(i)
    verbrelation=''
    real_verb=''
    verbdict={}
    verbs_split=df.loc[line,3].split(' ')
    for verb in verblist:
        verbdict.update(verb)
    for indexx,one in enumerate(verbs_split):
        if verbdict.get(one):
            verbrelation=one
            relations=sentence2relations.get(i)
            relations_display=[relation[-2] for relation in relations]
            verbprototype=sentence2verbprototype.get(i)
            verbprototype_dict={}
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation
                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation
                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            verbs_split[indexx]=verb_prototype
        elif bedict.get(one):
            verbs_split[indexx]=bedict.get(one)
    if bedict.get(' '.join(verbs_split[:2])):
        final=' '.join(['be']+verbs_split[2:])
    else:
        final=' '.join(verbs_split)
    finallist.append(final)
df[17]=finallist

In [89]:
ind=1
for i in all_relations_list:
    names['sentence'+str(ind)]=i
    ind+=1

In [90]:
ind

40215

In [91]:
sentence689

'Stroke is the leading cause of death in China, and predicting the stroke burden could provide essential information guiding the setting of medium- and long-term health policies and priorities.'

In [92]:
relation_dict_copy=copy.deepcopy(all_relations)

In [93]:
len(relation_dict_copy)

40214

In [101]:
suc=[]
for i in range(1,len(relation_dict_copy)+1):
    if relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        openie=[]
        for j in relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
            j1=j.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            j2=j.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(j1,df.iloc[i-1,1]) and re.search(j2,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target1'),df.iloc[i-1,-1],j.get('target2')]]
                openie.append([j.get('target1'),df.iloc[i-1,-1],j.get('target2')])
            elif re.search(j2,df.iloc[i-1,1]) and re.search(j1,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target2'),df.iloc[i-1,-1],j.get('target1')]]
                openie.append([j.get('target2'),df.iloc[i-1,-1],j.get('target1')]) 
        if not openie:
            suc.append(names[df.iloc[i-1,0][9:-4]])    

In [102]:
error=[]
for i in range(1,len(relation_dict_copy)+1):
    if not relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        error.append(names[df.iloc[i-1,0][9:-4]])

In [103]:
len(suc)

37250

In [104]:
len(relation_dict_copy)

40214

In [105]:
len(error)

0

## Deal with AllenNLP

In [70]:
sys.path.append("..")
PREDICTOR = Predictor.from_path('../openie-model.2020.03.26.tar.gz')

In [100]:
def clean(sent):
    return re.sub(' +', ' ', sent.replace('\r', ' ').replace('\n', ' '))
def get_tags_allen(sent):
    sent = sent.replace('[', '(').replace(']', ')')
    res = PREDICTOR.predict(
      sentence=sent
    )
    return res
def parse_allen(ss):
    words = ss['words']
    res = []
    for v in ss['verbs']:
        tags = v['tags']
        d = {}
        for i in range(len(words)):
            t = tags[i].replace('I-', '').replace('B-', '')
            w = words[i]
            if t in d:
                d[t] += ' '+ w
            else:
                d[t] = w
        res.append(d)
    return res

In [99]:
sentence2allennlp={}
for i in relation_dict_copy.keys():
    res=get_tags_allen(i)
    ress=parse_allen(res)
    sentence2allennlp[i]=ress

In [106]:
for i,j in relation_dict_copy.items():
    for k in j:
        k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        for l in sentence2allennlp.get(i):
            if l.get('ARG1') and l.get('ARG2') and l.get('V'):
                if re.search(k1,l.get('ARG1')) and re.search(k2,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target1'), l.get('V') ,k.get('target2')]]
                elif re.search(k2,l.get('ARG1')) and re.search(k1,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target2'), l.get('V') ,k.get('target1')]]

In [107]:
for i,j in relation_dict_copy.items():
    for k in j:
        if k.get('allennlp'):
            print(k)
            break
    else:
        continue
    break

{'PMID': '27253662', 'entity1': 'MALT1', 'entity2': 'Immunologic Deficiency Syndromes', 'sentence': 'CONCLUSIONS: Along with FOXP3, interleukin 2 receptor alpha chain (IL2RA), and cytotoxic T-lymphocyte protein 4 precursor (CTLA-4) mutations, MALT1 deficiency should now be considered as a possible cause of IPEX-like syndrome associated with immunodeficiency that can be cured by hematopoietic stem cell transplantation.', 'AB': 'OBJECTIVE: Early-onset inflammatory bowel diseases can result from a wide spectrum of rare mendelian disorders. Early molecular diagnosis is crucial in defining treatment and in improving life expectancy. Herein we aimed at defining the mechanism of an immunodeficiency-polyendrocrinopathy and enteropathy-X-linked (IPEX)-like disease combined with a severe immunodeficiency in 2 siblings born from distantly related parents. METHODS: Whole exome sequencing was performed on blood-extracted genomic DNA from the 2 affected children and their parents on the genomic plat

In [108]:
len(sentence2adv)

40214

In [109]:
negativewords=['no','not','never','hardly','barely','scarcely','rarely','few','little','seldom','neither','nor']
negativewords_dict={}
for i in negativewords:
    negativewords_dict[i]=i

In [110]:
def work(i,j,triple,abnormal, negative_sentences,networkx_raw):
    adv=sentence2adv.get(i)
    words=sentence2words.get(i)
    sentence=i
    relations=sentence2relations.get(i)
    relations_display=[relation[-2] for relation in relations]
    verblist=sentence2verblist.get(i)
    verbprototype=sentence2verbprototype.get(i)
    verbprototype_dict={}
    for verbp in verbprototype:
        verbprototype_dict.update(verbp)
    G = nx.Graph()
    for word in words:
        G.add_node(word)
    G.add_node('root')
    for r in range(len(relations)):
        if relations[r][4]=='root':
            G.add_edge(relations[r][0], 'root',labels=relations[r][4])
        else:
            G.add_edge(relations[r][0], relations[r][1],labels=relations[r][4])
    for k in j:
        if verblist:
            k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(k1,sentence) and re.search(k2,sentence):
                worddict={}
                for word in words:
                    worddict[word]=word
                if re.search(k1,sentence).span()[0] < re.search(k2,sentence).span()[0]:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k1_word)[-1], target=re.split('[ -]',k2_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one 
                        if verbrelation :
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target1'),verb_prototype,k.get('target2')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target1'),'former_entity':k.get('entity1'),'former_entity_type':k.get('target1_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target2'),
                                                   'latter_entity':k.get('entity2'),'latter_entity_type':k.get('target2_type')})
                                    k['networkx']=[k.get('target1'),verb_prototype,k.get('target2')]
                                else:
                                    negative_sentences.append(i)
                else:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k2_word)[-1], target=re.split('[ -]',k1_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one
                        if verbrelation:
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target2'),verb_prototype,k.get('target1')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target2'),'former_entity':k.get('entity2'),'former_entity_type':k.get('target2_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target1'),
                                                   'latter_entity':k.get('entity1'),'latter_entity_type':k.get('target1_type')})
                                    k['networkx']=[k.get('target2'),verb_prototype,k.get('target1')]
                                else:
                                    negative_sentences.append(i)
        networkx_raw.append(k)
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    triple=manager.list()
    abnormal=manager.list()
    negative_sentences=manager.list()
    networkx_raw=manager.list()
    for i,j in relation_dict_copy.items():
        p.apply_async(work, args=(i,j,triple,abnormal,negative_sentences,networkx_raw))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 103567.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 276s


In [111]:
triple_list=list(triple)
abnormal_list=list(abnormal)
negative_list=list(negative_sentences)
networkx_raw_list=list(networkx_raw)
networkx_raw_dict={}
for i in networkx_raw_list:
    networkx_raw_dict[i.get('sentence')]=networkx_raw_dict.get(i.get('sentence'),[])+[i]

In [112]:
pickle.dump(triple_list,open('results/step3/triple_list.pkl','wb'))

In [113]:
pickle.dump(abnormal_list,open('results/step3/abnormal_list.pkl','wb'))
pickle.dump(negative_list,open('results/step3/negative_list.pkl','wb'))
pickle.dump(networkx_raw_list,open('results/step3/networkx_raw_list.pkl','wb'))
pickle.dump(networkx_raw_dict,open('results/step3/networkx_raw_dict.pkl','wb'))

In [114]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [115]:
networkx_raw_list[0]

{'PMID': '37116731',
 'entity1': 'SMAD7',
 'entity2': 'PRKN',
 'sentence': 'CONCLUSIONS: We demonstrate that miR-181c levels are significantly increased in frail elderly adults with DM and HFpEF and that miR-181c targets PRKN and SMAD7 in human cardiac fibroblasts.',
 'AB': 'BACKGROUND: Cardiac fibrosis represents a key element in the pathophysiology of heart failure with preserved ejection fraction (HFpEF), a condition highly prevalent amongst geriatric patients, especially if diabetic. The microRNA 181c (miR-181c) has been shown to be associated with the response to exercise training in HFpEF patients and has been also linked to diabetic cardiovascular complications. However, the underlying mechanisms have not been fully elucidated. OBJECTIVE: To measure circulating miR-181c in elderly patients with HFpEF and diabetes mellitus (DM) and identify gene targets pathophysiologically relevant in HFpEF. METHODS: We quantified circulating miR-181c in frail older adults with a confirmed diagn

In [116]:
len(networkx_raw_dict)

39887

In [117]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i

In [118]:
for i in networkx_raw_list:
    openie_mid=[]
    openie_final=[]
    if i.get('openie'):
        for j in i.get('openie'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                openie_mid.append(j)
        adv=sentence2adv.get(i.get('sentence_transformed'))
        adv_dict={}
        for l in adv:
            adv_dict.update(l)
        for k in openie_mid:
            for m in k[1].split(' '):
                if adv_dict.get(m):
                    break
            else:
                if k not in openie_final:
                    openie_final.append(k)
    i['openie_final']=openie_final

In [119]:
for i in networkx_raw_list:
    allennlp_final=[]
    if i.get('allennlp'):
        for j in i.get('allennlp'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                allennlp_final.append(j)
    i['allennlp_final']=allennlp_final

In [120]:
for i in networkx_raw_list:
    if i.get('networkx'):
        if bedict.get(i.get('networkx')[1]):
            i['networkx']=''
        elif i.get('networkx')[1]=='"':
            i['networkx']=''
    else:
        i['networkx']=''

In [121]:
networkx_raw_final=[]
for i in networkx_raw_list:
    if i.get('networkx') or i.get('openie_final') or i.get('allennlp_final'):
        networkx_raw_final.append(i)

In [122]:
len(networkx_raw_final)

26512

In [123]:
for i in networkx_raw_final:
    bidirection=False
    if i.get('networkx'):
        if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
            i['bidirection']=True
            continue
    if i.get('allennlp'):
        for j in i.get('openie_final'):
            if j=='associated' or j=='associate' :
                bidirection=True
                break
    if i.get('openie_final'):
        for j in i.get('openie_final'):
            for k in j[1].split(' '):
                if k=='associated' or  k=='associate' :
                    bidirection=True
                    break
            else:
                continue
            break
    i['bidirection']=bidirection

In [124]:
Relation_raw=copy.deepcopy(networkx_raw_final)

In [125]:
pickle.dump(networkx_raw_final,open('results/step3/networkx_raw_final.pkl','wb'))

In [126]:
networkx_raw_final[0]

{'PMID': '32807694',
 'entity1': 'ZPR1',
 'entity2': 'LDLR',
 'sentence': 'CONCLUSIONS: The cardiovascular risk in patients with FH is highly heterogeneous, and this study suggests that the rs964184 variant of the ZPR1 gene represents one of the important modulating factors.',
 'AB': 'BACKGROUND: The rs964184 variant in the ZPR1 gene has been associated with blood lipids and cardiovascular disease risk in the general population through genome-wide association study, but its effect in patients with familial hypercholesterolemia (FH) has never been studied. OBJECTIVE: The objectives of the present study are to investigate the effect of the rs964184 SNP on blood lipids and on the risk of incident myocardial infarction (MI) in patients with FH. METHODS: This study included 725 patients with genetically confirmed FH. The MI events that occurred throughout the lifespan until the last medical visit were included. The median observation period was 50 years. An exome chip genotyping method (Ill

In [127]:
len(networkx_raw_final)

26512

In [128]:
networkx_raw_final[0]

{'PMID': '32807694',
 'entity1': 'ZPR1',
 'entity2': 'LDLR',
 'sentence': 'CONCLUSIONS: The cardiovascular risk in patients with FH is highly heterogeneous, and this study suggests that the rs964184 variant of the ZPR1 gene represents one of the important modulating factors.',
 'AB': 'BACKGROUND: The rs964184 variant in the ZPR1 gene has been associated with blood lipids and cardiovascular disease risk in the general population through genome-wide association study, but its effect in patients with familial hypercholesterolemia (FH) has never been studied. OBJECTIVE: The objectives of the present study are to investigate the effect of the rs964184 SNP on blood lipids and on the risk of incident myocardial infarction (MI) in patients with FH. METHODS: This study included 725 patients with genetically confirmed FH. The MI events that occurred throughout the lifespan until the last medical visit were included. The median observation period was 50 years. An exome chip genotyping method (Ill

In [129]:
collections_Relation_raw=db.Relation_raw

In [130]:
now_time

'2023_09_30'

In [131]:
for i in Relation_raw:
    i['addtime']=now_time

In [132]:
collections_Relation_raw.insert_many(Relation_raw)

In [133]:
Relation_raw_all_with_disease=copy.deepcopy(Relation_raw)

In [134]:
len(Relation_raw_all_with_disease)

26512

In [135]:
Relation_raw_all_with_disease[10]

{'PMID': '29361001',
 'entity1': 'BRCA2',
 'entity2': 'RAD51C',
 'sentence': 'Methods: A decision-analytic model comparing lifetime costs and effects of criteria/FH-based BRCA1/BRCA2 testing is compared with BRCA1/BRCA2/RAD51C/RAD51D/BRIP1/PALB2 testing in those fulfilling clinical criteria/strong FH of cancer (>=10% BRCA1/BRCA2 probability) and all women age 30 years or older.',
 'AB': "Background: The cost-effectiveness of population-based panel testing for high- and moderate-penetrance ovarian cancer (OC)/breast cancer (BC) gene mutations is unknown. We evaluate the cost-effectiveness of population-based BRCA1/BRCA2/RAD51C/RAD51D/BRIP1/PALB2 mutation testing compared with clinical criteria/family history (FH) testing in unselected general population women. Methods: A decision-analytic model comparing lifetime costs and effects of criteria/FH-based BRCA1/BRCA2 testing is compared with BRCA1/BRCA2/RAD51C/RAD51D/BRIP1/PALB2 testing in those fulfilling clinical criteria/strong FH of can

In [136]:
all_es=json.load(open('results/step2/all_es_lack.json'))

In [137]:
multi_chemical_type2={}
for i in all_es:
    if not multi_chemical_type2.get(i.get('entity')):
        multi_chemical_type2[i.get('entity')]=[i.get('type')]
    else:
        if i.get('type') not in multi_chemical_type2.get(i.get('entity')):
            multi_chemical_type2[i.get('entity')]+=[i.get('type')]

In [138]:
cxz=0
for i,j in multi_chemical_type2.items():
    if len(j)>1:
        cxz+=1
        print({i:j})
print(cxz)

{'Lipopolysaccharides': ['Carbohydrate', 'Lipid', 'Toxin']}
{'Vancomycin': ['Carbohydrate', 'Peptide']}
{'Ceramides': ['Carbohydrate', 'Lipid']}
{'Sphingomyelins': ['Carbohydrate', 'Lipid']}
{'Bleomycin': ['Carbohydrate', 'Peptide']}
{'Teicoplanin': ['Carbohydrate', 'Peptide']}
{'Lipid A': ['Carbohydrate', 'Lipid', 'Toxin']}
{'G(M1) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Gangliosides': ['Carbohydrate', 'Lipid']}
{'Glycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Sulfoglycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Glycolipids': ['Carbohydrate', 'Lipid']}
{'G(M3) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Glycopeptides': ['Carbohydrate', 'Peptide']}
{'Glycosylphosphatidylinositols': ['Carbohydrate', 'Lipid']}
{'Globosides': ['Carbohydrate', 'Lipid']}
{'Ristocetin': ['Carbohydrate', 'Peptide']}
{'Glucosylceramides': ['Carbohydrate', 'Lipid']}
{'Holothurin': ['Carbohydrate', 'Toxin']}
{'Glycerylphosphorylcholine': ['Carbohydrate', 'Lipid']}
{'Glycerophosphates': ['Carbohydrate', 'L

In [139]:
Relation_raw_all_final=[]
for i in Relation_raw_all_with_disease:
    if multi_chemical_type2.get(i.get('entity1')) and multi_chemical_type2.get(i.get('entity2')):
        for j in multi_chemical_type2.get(i.get('entity1')):
            for k in multi_chemical_type2.get(i.get('entity2')):
                mid=copy.deepcopy(i)
                mid.update({'target1_type':j,'target2_type':k})
                Relation_raw_all_final.append(mid)
    else:
        Relation_raw_all_final.append(i)
        if multi_chemical_type2.get(i.get('entity1')):
            for j in multi_chemical_type2.get(i.get('entity1')):
                if i.get('target1_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target1_type':j})
                    Relation_raw_all_final.append(mid)
        elif multi_chemical_type2.get(i.get('entity2')):
            for j in multi_chemical_type2.get(i.get('entity2')):
                if i.get('target2_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target2_type':j})
                    Relation_raw_all_final.append(mid)

In [140]:
len(Relation_raw_all_final)

26810

In [141]:
direction=set()
direction_list=[]
direction_type=[]
for i in Relation_raw_all_final:
    if i.get('openie_final'):
        if i.get('openie_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('networkx'):
        if i.get('networkx')[0]!= i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('allennlp_final'):
        if i.get('allennlp_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    if i.get('bidirection'):
        if (i.get('entity2'),(i.get('entity1'))) not in direction :
            direction.add((i.get('entity2'),(i.get('entity1'))))
            direction_list.append((i.get('entity2'),(i.get('entity1'))))
            direction_type.append((i.get('target2_type'),(i.get('target1_type'))))
        if (i.get('entity1'),(i.get('entity2'))) not in direction:
            direction.add((i.get('entity1'),(i.get('entity2'))))
            direction_list.append((i.get('entity1'),(i.get('entity2'))))
            direction_type.append((i.get('target1_type'),(i.get('target2_type'))))
    else:
        direction.add((i.get('entity1'),(i.get('entity2'))))
        direction_list.append((i.get('entity1'),(i.get('entity2'))))
        direction_type.append((i.get('target1_type'),(i.get('target2_type'))))

In [142]:
Relation_raw_all_final[10]

{'PMID': '29361001',
 'entity1': 'BRCA2',
 'entity2': 'RAD51C',
 'sentence': 'Methods: A decision-analytic model comparing lifetime costs and effects of criteria/FH-based BRCA1/BRCA2 testing is compared with BRCA1/BRCA2/RAD51C/RAD51D/BRIP1/PALB2 testing in those fulfilling clinical criteria/strong FH of cancer (>=10% BRCA1/BRCA2 probability) and all women age 30 years or older.',
 'AB': "Background: The cost-effectiveness of population-based panel testing for high- and moderate-penetrance ovarian cancer (OC)/breast cancer (BC) gene mutations is unknown. We evaluate the cost-effectiveness of population-based BRCA1/BRCA2/RAD51C/RAD51D/BRIP1/PALB2 mutation testing compared with clinical criteria/family history (FH) testing in unselected general population women. Methods: A decision-analytic model comparing lifetime costs and effects of criteria/FH-based BRCA1/BRCA2 testing is compared with BRCA1/BRCA2/RAD51C/RAD51D/BRIP1/PALB2 testing in those fulfilling clinical criteria/strong FH of can

In [143]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']
names=locals()
for i in rules_pairs:
    names[i]={}
    for k in ['./results/step3/rules/files/'+i+str(j)+'.pkl' for j in range(1,6)]:
        names[i].update(pickle.load(open(k,'rb')))

In [144]:
Relation_raw_all_final_copy=copy.deepcopy(Relation_raw_all_final)

In [145]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i
countt=0
counttt=0
fdas=[]
for i in Relation_raw_all_final_copy:
    del_index=set()
    if i.get('sentence_transformed'):
        verblist=sentence2verblist.get(i.get('sentence_transformed'))
    else:
        verblist=sentence2verblist.get(i.get('sentence'))
    verbrelation=''
    real_verb=''
    verbdict={}
    for verb in verblist:
        verbdict.update(verb)
    for indexx,j in enumerate(i.get('openie_final')):
        one=j[1]
        negative=False
        if i.get('sentence_transformed'):
            relations=sentence2relations.get(i.get('sentence_transformed'))
            verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
        else:
            relations=sentence2relations.get(i.get('sentence'))
            verbprototype=sentence2verbprototype.get(i.get('sentence'))
        for p in one.split(' '):
            for relation in relations:
                if relation[1]==p and negativewords_dict.get(relation[0]):
                    negative=True
            if verbdict.get(p):
                verbrelation=p
                relations_display=[relation[-2] for relation in relations]
                verbprototype_dict={}
                for verbp in verbprototype:
                    verbprototype_dict.update(verbp)
                for index_passive,passive in enumerate(relations_display):
                    if passive=='aux:pass':
                        if relations[index_passive][1]==verbrelation:
                            verb_prototype=verbrelation
                            break
                    elif passive=='acl' and relations[index_passive][-1]=='VBN':
                        if relations[index_passive][0]==verbrelation:
                            verb_prototype=verbrelation
                            break
                else:
                    for v in verbprototype:
                        if v.get(verbrelation):
                            verb_prototype=v.get(verbrelation)
                            break
                if verb_prototype!=one:
                    if countt<10:
                        countt+=1
                if not negative:
                    i['openie_final'][indexx][1]=verb_prototype
            elif bedict.get(one):
                if i.get('sentence_transformed'):
                    relations=sentence2relations.get(i.get('sentence_transformed'))
                    verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
                else:
                    relations=sentence2relations.get(i.get('sentence'))
                    verbprototype=sentence2verbprototype.get(i.get('sentence'))
                for relation in relations:
                    if relation[1]==one and negativewords_dict.get(relation[0]):
                        negative=True
                if not negative:
                    i['openie_final'][indexx][1]=bedict.get(one)
        if negative:
            fdas.append(i)
            i['openie_final'][indexx]=[]
    final=[]
    for t in i.get('openie_final'):
        if t != []:
            final.append(t)
    i['openie_final']=final

In [146]:
len(fdas)

2

In [147]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i
countt=0
counttt=0
fdas=[]
for i in Relation_raw_all_final_copy:
    if i.get('sentence_transformed'):
        verblist=sentence2verblist.get(i.get('sentence_transformed'))
    else:
        verblist=sentence2verblist.get(i.get('sentence'))
    verbrelation=''
    real_verb=''
    verbdict={}
    for verb in verblist:
        verbdict.update(verb)
    for indexx,j in enumerate(i.get('allennlp_final')):
        one=j[1]
        negative=False
        if i.get('sentence_transformed'):
            relations=sentence2relations.get(i.get('sentence_transformed'))
            verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
        else:
            relations=sentence2relations.get(i.get('sentence'))
            verbprototype=sentence2verbprototype.get(i.get('sentence'))
        for relation in relations:
            if relation[1]==one and negativewords_dict.get(relation[0]):
                negative=True
        if verbdict.get(one):
            verbrelation=one
            relations_display=[relation[-2] for relation in relations]
            verbprototype_dict={}
            for relation in relations:
                if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                    negative=True
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation
                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation
                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            if verb_prototype!=one:
                if countt<10:
                    countt+=1
            if not negative:
                i['allennlp_final'][indexx][1]=verb_prototype
        elif bedict.get(one):
            if i.get('sentence_transformed'):
                relations=sentence2relations.get(i.get('sentence_transformed'))
                verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
            else:
                relations=sentence2relations.get(i.get('sentence'))
                verbprototype=sentence2verbprototype.get(i.get('sentence'))
            for relation in relations:
                if relation[1]==one and negativewords_dict.get(relation[0]):
                    negative=True
            if not negative:
                i['allennlp_final'][indexx][1]=bedict.get(one)
        if negative:
            fdas.append(i)
            i['allennlp_final'][indexx]=[]
    final=[]
    for t in i.get('allennlp_final'):
        if t != []:
            final.append(t)
    i['allennlp_final']=final

In [148]:
len(fdas)

119

In [149]:
len(Relation_raw_all_final_copy)

26810

In [150]:
Relation_raw_all_final_copys=[]
for i in Relation_raw_all_final_copy:
    if i.get('networkx') or i.get('openie_final') or i.get('allennlp_final'):
        Relation_raw_all_final_copys.append(i)
print(len(Relation_raw_all_final_copys))

26699


In [151]:
for i in Relation_raw_all_final_copys:
    if i.get('_id'):
        del i['_id']
json.dump(Relation_raw_all_final_copys,open('results/step3/triplelist.json','w'))

In [152]:
len(Relation_raw_all_final_copys)

26699

In [153]:
collections_Relations_final=db.Relations_final

In [154]:
for i in Relation_raw_all_final_copys:
    i['addtime']=now_time

In [155]:
collections_Relations_final.insert_many(Relation_raw_all_final_copys)

In [156]:
pickle.dump(Relation_raw_all_final_copys,open('results/step3/Relation_raw_all_final_copy.pkl','wb'))

In [157]:
pickle.dump(Relation_raw,open('results/step3/Relation_raw.pkl','wb'))

# Below are dropped

# Deal with colons

## Prepare for OpenIE

In [59]:
os.makedirs('results/step3/openie_colon/filelist/')

In [60]:
ind=1
for i in sentence_colons.keys():
    f=open('results/step3/openie_colon/filelist/sentence'+str(ind)+'.txt','w')
    f.write(i)
    f.close()
    ind+=1

In [61]:
f=open('results/step3/openie_colon/filelist.txt','a')
for i in range(1,len(sentence_colons)+1):
    f.write('filelist/sentence'+str(i)+'.txt'+'\n')
f.close()

### Run in shell 

In [40]:
# cd results/step3/openie_colon
# nohup java -mx500g -cp ../../../../stanza_corenlp/stanford-corenlp-4.3.2.jar:../../../../stanza_corenlp/stanford-corenlp-4.3.2-models.jar:../../../../stanza_corenlp/CoreNLP-to-HTML.xsl:../../../../slf4j-api.jar:../../../../stanza_corenlp/slf4j-simple.jar edu.stanford.nlp.naturalli.OpenIE -format reverb -filelist ./filelist.txt -output ./openie_colon.txt > ./openie_colon.log &
# cd ../../../

## Prepare for Stanza

In [63]:
os.makedirs('results/step3/openie_colon/files/')
k=1
sentence_colons_list=list(sentence_colons.keys())
for i in range(0,len(sentence_colons_list),len(sentence_colons_list)//4):
    pickle.dump(sentence_colons_list[i:i+len(sentence_colons_list)//4],open('results/step3/openie_colon/files/openie_colon'+str(k)+'.pkl','wb'))
    k+=1

FileExistsError: [Errno 17] File exists: 'results/step3/openie_colon/files/'

In [65]:
k=1
for i in range(0,len(sentence_colons_list),len(sentence_colons_list)//4):
    f=open('results/step3/rules_colon/files/rules_colon'+str(k)+'.py','w')
    f.write('''
import stanza
import os
import datetime
import time
nlp = stanza.Pipeline('en', package='genia',download_method=None)
import pickle
raw=pickle.load(open('../../openie_colon/files/openie_colon'''+str(k)+'''.pkl','rb'))
import re 

def nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbtype,sentence2adv):
    sentence=i
    doc=nlp(sentence)
    words=[]
    for sent in doc.sentences:
        for word in sent.words:
            words.append(word.text)
    relations=[]
    verblist=[]
    verbprototype=[]
    adv=[]
    for sent in doc.sentences:
        for word in sent.words:
            relations.append([word.text,words[int(word.head)-1],int(word.id),int(word.head),word.deprel,word.xpos])
            if word.xpos in ['VB','VBD','VBG','VBN', 'VBP', 'VBZ']:
                verblist.append({word.text:word.xpos})
                verbprototype.append({word.text:word.lemma})
            if word.xpos in ['RB','RBR','RBS']:
                adv.append({word.text:word.xpos})
    sentence2words[i]=words
    sentence2relations[i]=relations
    sentence2verblist[i]=verblist
    sentence2verbprototype[i]=verbprototype
    sentence2adv[i]=adv


start=datetime.datetime.now()
print('Parent process %s.' % os.getpid())
sentence2words={}
sentence2relations={}
sentence2verblist={}
sentence2verbprototype={}
sentence2adv={}
for i in raw:
    nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbprototype,sentence2adv)
pickle.dump(sentence2words,open('sentence2words'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2relations,open('sentence2relations'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verblist,open('sentence2verblist'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verbprototype,open('sentence2verbprototype'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2adv,open('sentence2adv'''+str(k)+'''.pkl','wb'))
print('Waiting for all subprocesses done...')

end=datetime.datetime.now()
print("The running time is "+str((end-start).seconds)+"s")  
    ''')
    f.close()
    k+=1

### Run in shell 

In [111]:
# cd results/step3/rules_colon/files
# for i in {1..5}; do echo "nohup python rules_colon${i}.py >> rules_colon.log &" | bash; done
# cd ../../../../

## Deal with OpenIE

In [66]:
openie_colon_table=pd.read_table('results/step3/openie_colon/openie_colon.txt',header=None)

In [67]:
openie_colon_table.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,filelist/sentence40.txt,0,miR 4500,was reduced in,induced HUVECs,4,7,7,10,12,14,1.0,ARG2 was upregulated and miR - 4500 was reduce...,NN VBD VBN CC NN HYPH CD VBD VBN IN NN HYPH VB...,mir 4500,be reduce in,induce huvec
1,filelist/sentence40.txt,0,ARG2,was,upregulated,0,1,1,2,2,3,1.0,ARG2 was upregulated and miR - 4500 was reduce...,NN VBD VBN CC NN HYPH CD VBD VBN IN NN HYPH VB...,arg2,be,upregulate
2,filelist/sentence40.txt,0,miR 4500,was reduced in,HUVECs,4,7,7,10,13,14,1.0,ARG2 was upregulated and miR - 4500 was reduce...,NN VBD VBN CC NN HYPH CD VBD VBN IN NN HYPH VB...,mir 4500,be reduce in,huvec
3,filelist/sentence40.txt,0,miR,was reduced in,H2O2 induced HUVECs,4,5,7,10,10,14,1.0,ARG2 was upregulated and miR - 4500 was reduce...,NN VBD VBN CC NN HYPH CD VBD VBN IN NN HYPH VB...,mir,be reduce in,h2o2 induce huvec
4,filelist/sentence40.txt,0,miR 4500,was,reduced,4,7,7,8,8,9,1.0,ARG2 was upregulated and miR - 4500 was reduce...,NN VBD VBN CC NN HYPH CD VBD VBN IN NN HYPH VB...,mir 4500,be,reduce


In [68]:
len(openie_colon_table)

561

In [69]:
df=openie_colon_table.loc[:,[0,2,3,4,12]]

In [70]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'

In [84]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']
for i in rules_pairs:
    names[i]={}
    for k in ['results/step3/rules_colon/files/'+i+str(j)+'.pkl' for j in range(1,6)]:
        names[i].update(pickle.load(open(k,'rb')))

In [88]:
finallist=[]
for line in range(len(df)):
    i=sentence_colons_list[int(df.iloc[line,0][17:-4])-1]
    verblist=sentence2verblist.get(i)
    verbrelation=''
    real_verb=''
    verbdict={}
    verbs_split=df.loc[line,3].split(' ')
    for verb in verblist:
        verbdict.update(verb)
    for indexx,one in enumerate(verbs_split):
        if verbdict.get(one):
            verbrelation=one
            relations=sentence2relations.get(i)
            relations_display=[relation[-2] for relation in relations]
            verbprototype=sentence2verbprototype.get(i)
            verbprototype_dict={}
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation
                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation
                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            verbs_split[indexx]=verb_prototype
        elif bedict.get(one):
            verbs_split[indexx]=bedict.get(one)
    if bedict.get(' '.join(verbs_split[:2])):
        final=' '.join(['be']+verbs_split[2:])
    else:
        final=' '.join(verbs_split)
    finallist.append(final)
df[17]=finallist

In [124]:
ind=1
for i in sentence_colons_list:
    names['sentence'+str(ind)]=i
    ind+=1

In [170]:
sentence1

'There was association between folate pathway gene polymorphism of Solute Carrier Family 19 Member 1 (SLC19A1) and Methylenetetrahydrofolate Reductase (MTHFR) with erythrocyte SOD and GSH-PX activities, and folate levels in male but not in female aging subjects.'

In [171]:
ind

687

In [91]:
relation_dict_copy=copy.deepcopy(sentence_colons)

In [92]:
len(relation_dict_copy)

50

In [112]:
names[df.iloc[i-1,0][9:-4]]

'The increased IGF-1 in Panx1-knockdown ECFCs was abrogated, respectively, by inhibitors of FAK (PF562271), ERK (U0126), and STAT3 (NSC74859) and supplemented calcium.'

In [123]:
list(relation_dict_copy.items())[0]

('There was association between folate pathway gene polymorphism of Solute Carrier Family 19 Member 1 (SLC19A1) and Methylenetetrahydrofolate Reductase (MTHFR) with erythrocyte SOD and GSH-PX activities, and folate levels in male but not in female aging subjects.',
 [{'PMID': '37201763',
   'entity1': 'SLC19A1',
   'entity2': 'MTHFR',
   'sentence': 'CONCLUSIONS: There was association between folate pathway gene polymorphism of Solute Carrier Family 19 Member 1 (SLC19A1) and Methylenetetrahydrofolate Reductase (MTHFR) with erythrocyte SOD and GSH-PX activities, and folate levels in male but not in female aging subjects.',
   'AB': "BACKGROUND: Plasma folate levels are closely related to antioxidant capacity and are regulated by folate pathway gene polymorphism. However, few studies have explored the gender-specific association of folate pathway gene polymorphism with oxidative stress biomarkers. The present study was designed to explore the gender-specific independent and combined impa

In [121]:
names[df.iloc[0,0][9:-4]]

'In skin photoaging samples, PVT1 and AQP3 were poorly expressed, while miR-551b-3p was highly expressed.'

In [122]:
relation_dict_copy.get(names[df.iloc[0,0][9:-4]])

In [127]:
suc=[]
for i in range(1,len(relation_dict_copy)+1):
    if relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        openie=[]
        for j in relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
            j1=j.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            j2=j.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(j1,df.iloc[i-1,1]) and re.search(j2,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target1'),df.iloc[i-1,-1],j.get('target2')]]
                openie.append([j.get('target1'),df.iloc[i-1,-1],j.get('target2')])
            elif re.search(j2,df.iloc[i-1,1]) and re.search(j1,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target2'),df.iloc[i-1,-1],j.get('target1')]]
                openie.append([j.get('target2'),df.iloc[i-1,-1],j.get('target1')]) 
        if not openie:
            suc.append(names[df.iloc[i-1,0][9:-4]])    

In [129]:
error=[]
for i in range(1,len(relation_dict_copy)+1):
    if not relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        error.append(names[df.iloc[i-1,0][9:-4]])

In [130]:
len(suc)

45

In [131]:
len(relation_dict_copy)

50

In [132]:
len(error)

0